In [93]:
import pandas as pd

def import_data():
    gtfs_dir = '/Users/denisvoroncov/Desktop/Project/GTFS_OP_2024_obb/'

    # Load data
    agency_df = pd.read_csv(gtfs_dir + 'agency.txt')
    stops_df = pd.read_csv(gtfs_dir + 'stops.txt')
    routes_df = pd.read_csv(gtfs_dir + 'routes.txt')
    trips_df = pd.read_csv(gtfs_dir + 'trips.txt')
    stop_times_df = pd.read_csv(gtfs_dir + 'stop_times.txt')
    calendar_df = pd.read_csv(gtfs_dir + 'calendar.txt')
    calendar_dates_df = pd.read_csv(gtfs_dir + 'calendar_dates.txt')
    shapes_df = pd.read_csv(gtfs_dir + 'shapes.txt')
    
    # Return all DataFrames
    return agency_df, stops_df, routes_df, trips_df, stop_times_df, calendar_df, calendar_dates_df, shapes_df

# Import and display data
agency, stops, routes, trips, stop_times, calendar, calendar_dates, shapes = import_data()

In [94]:
import scipy.stats as stats
3

# Transfer reliability 

def calculate_transfer_probability(transfer_window, shape=2, scale=3):
    
    # transfer_window = next_bus_departure - scheduled_arrival
    # if transfer_window < 0:
    #     return 0.0  # No chance of a successful transfer if arrival is after departure
    return stats.gamma.cdf(transfer_window, a=shape, scale=scale)

In [95]:
import sys
import heapq
from collections import defaultdict
from datetime import datetime, timedelta
from scipy.stats import norm

# Hilfsfunktion: Zeit in Minuten umwandeln
def time_to_minutes(time_str):
    hours, minutes, seconds = map(int, time_str.split(":"))
    return hours * 60 + minutes + seconds / 60

def minutes_to_time(minutes):
    hours = int(minutes // 60)
    minutes = int(minutes % 60)
    return f"{hours:02d}:{minutes:02d}"

def get_weekday(date):
    weekdays = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"]
    return weekdays[date.weekday()]

from datetime import datetime


#FFFFFFFIIIIIIIIXXXXXXEEEEEEDDDDDD

def is_service_available(service_id, date, calendar, calendar_dates):

    # Convert date to string in YYYYMMDD format
    date_str = date.strftime("%Y%m%d")
    weekday = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"][date.weekday()]

    # Step 1: Check for exceptions in calendar_dates
    if service_id in calendar_dates["service_id"].values:
        exceptions = calendar_dates[calendar_dates["service_id"] == service_id]
        for _, exception in exceptions.iterrows():
            if exception["date"] == int(date_str):
                if exception["exception_type"] == 2:  # Service is added as an exception
                    return True
                elif exception["exception_type"] == 1:  # Service is removed as an exception
                    return False

    # Step 2: Check for regular service in calendar
    if service_id in calendar["service_id"].values:
        service = calendar[calendar["service_id"] == service_id]
        # Check if date is within start_date and end_date
        if int(service["start_date"].iloc[0]) <= int(date_str) <= int(service["end_date"].iloc[0]):
            # Check if the service operates on this weekday
            is_available = (service[weekday].iloc[0] == 1)
            return is_available



# THIS ONE HAS BUGS
def identify_transfer_stops(stop_times, trips, date, calendar, calendar_dates, stops):
    """
    Identify transfer stops and print all remaining routes and the stops they serve after filtering.
    
    Parameters:
        stop_times (pd.DataFrame): Stop times DataFrame.
        trips (pd.DataFrame): Trips DataFrame.
        date (datetime.date): Date to filter services.
        calendar (pd.DataFrame): Calendar DataFrame for regular services.
        calendar_dates (pd.DataFrame): Calendar dates DataFrame for exceptions.
        stops (pd.DataFrame): Stops DataFrame for mapping stop IDs to names.
    
    Returns:
        set: Set of transfer stop IDs.
    """
    # Map trip_id to route_id and service_id
    trip_to_route = trips.set_index("trip_id")["route_id"].to_dict()
    trip_to_service = trips.set_index("trip_id")["service_id"].to_dict()

    # Add route_id and service_id to stop_times
    stop_times = stop_times.copy()
    stop_times["route_id"] = stop_times["trip_id"].map(trip_to_route)
    stop_times["service_id"] = stop_times["trip_id"].map(trip_to_service)

    # Precompute service availability
    unique_service_ids = stop_times["service_id"].unique()
    service_availability = {
        service_id: is_service_available(service_id, date, calendar, calendar_dates)
        for service_id in unique_service_ids
    }

    # Filter stop_times for active services
    stop_times = stop_times[stop_times["service_id"].map(service_availability)]

    # Group by stop_id and count unique route_ids
    stop_route_counts = stop_times.groupby("stop_id")["route_id"].nunique()

    # Identify transfer stops (stops with more than one route)
    transfer_stops = set(stop_route_counts[stop_route_counts > 1].index)

    # Map stop_id to stop_name
    stop_id_to_name = stops.set_index("stop_id")["stop_name"].to_dict()

    # Group by route_id and list stops
    grouped_routes = stop_times.groupby("route_id")["stop_id"].unique()

    # Print remaining routes and their stops
    for route_id, stop_ids in grouped_routes.items():
        #print(f"Route: {route_id}")
        stop_names = [stop_id_to_name.get(stop_id, f"Unknown Stop ({stop_id})") for stop_id in stop_ids]
        #print(f"  Stops: {', '.join(stop_names)}")

    return transfer_stops


# Simplify stop_times by collapsing non-transfer stops
def simplify_network_with_no_transfer_stops(start_stop_name, end_stop_name, stop_times, trips, transfer_stops, date, calendar, calendar_dates, stops): 
    

    # Robust mapping from stop name to stop ID
    if start_stop_name not in stops["stop_name"].values:
        raise ValueError(f"Start stop name '{start_stop_name}' not found in stops DataFrame.")
    if end_stop_name not in stops["stop_name"].values:
        raise ValueError(f"End stop name '{end_stop_name}' not found in stops DataFrame.")

    stop_name_to_stop_id = stops.set_index("stop_name")["stop_id"].to_dict()
    start_stop_id = stop_name_to_stop_id[start_stop_name]
    end_stop_id = stop_name_to_stop_id[end_stop_name]


    trip_id_to_service = trips.set_index("trip_id")["service_id"].to_dict()

    simplified_rows = []
    grouped = stop_times.groupby("trip_id")
    
    for trip_id, group in grouped: # group of stop in a particular trip
        service_id = trip_id_to_service[trip_id]
        if not is_service_available(service_id, date, calendar, calendar_dates):
            continue
        

        group = group.sort_values("stop_sequence")

        for _, row in group.iterrows(): # iterate through each row of sorted stops in a trip
            stop_id = row["stop_id"]

            if stop_id in transfer_stops or stop_id == start_stop_id or stop_id == end_stop_id:  # Handle transfer stops + start point + destination point
                # Add the current collapsed segment to the result
                simplified_rows.append(row)

    # Convert the simplified rows into a DataFrame
    simplified_stop_times = pd.DataFrame(simplified_rows)

    # Reset stop_sequence values to be consecutive for each trip_id
    simplified_stop_times["stop_sequence"] = simplified_stop_times.groupby("trip_id").cumcount() + 1

    return simplified_stop_times


In [96]:
def prepare_network(
    start_stop_name, 
    end_stop_name, 
    start_datetime, 
    time_budget_minutes, 
    stop_times, 
    trips, 
    calendar, 
    calendar_dates, 
    stops
):
    """
    Prepare the network by filtering stop_times based on the start time, time budget,
    and active services on the given date.
    """
    print(f"Rows before time window filter: {len(stop_times)}")

    # Parse the start datetime and calculate time window
    start_time_obj = datetime.strptime(start_datetime, "%Y-%m-%d %H:%M:%S")
    end_time_obj = start_time_obj + timedelta(minutes=time_budget_minutes)
    date = start_time_obj.date()
    stop_times_copy = stop_times.copy()
    # Filter out stops outside the time window
    stop_times_copy["arrival_minutes"] = stop_times_copy["arrival_time"].apply(time_to_minutes)
    stop_times_copy["departure_minutes"] = stop_times_copy["departure_time"].apply(time_to_minutes)

    start_minutes = time_to_minutes(start_time_obj.strftime("%H:%M:%S"))
    end_minutes = time_to_minutes(end_time_obj.strftime("%H:%M:%S"))

    stop_times_copy = stop_times_copy[
        (stop_times_copy["arrival_minutes"] >= start_minutes) &
        (stop_times_copy["departure_minutes"] <= end_minutes)
    ]

    print(f"Rows after time window filter: {len(stop_times_copy)}")

    # Precompute service availability
    trip_id_to_service = trips.set_index("trip_id")["service_id"].to_dict()
    # Ensure it's a new DataFrame, not a slice
    stop_times_copy["service_id"] = stop_times_copy["trip_id"].map(trip_id_to_service)

    unique_service_ids = stop_times_copy["service_id"].dropna().unique()
    service_availability = {
        service_id: is_service_available(service_id, date, calendar, calendar_dates)
        for service_id in unique_service_ids
    }

    # Count rows before availability filter
    before_filter_count = len(stop_times_copy)

    # Filter using precomputed availability
    # Use .map() with a default value directly to avoid NaN
    # Filter using precomputed availability
    stop_times_copy = stop_times_copy[
    stop_times_copy["service_id"].map(lambda x: service_availability.get(x, False)).astype(bool)
    ]

    # Count rows after availability filter
    after_filter_count = len(stop_times_copy)

    print(f"Rows before availability filter: {before_filter_count}")
    print(f"Rows after availability filter: {after_filter_count}")

    # Identify transfer stops
    #transfer_stops = identify_transfer_stops(stop_times_copy, trips, date, calendar, calendar_dates, stops)

    # Simplify the network (use your existing logic here)
    #simplified_stop_times = simplify_network_with_no_transfer_stops(
        #start_stop_name, 
        #end_stop_name, 
        #stop_times_copy, 
        #trips, 
        #transfer_stops, 
        #date, 
        #calendar, 
        #calendar_dates,
        #stops
    #)
    print(f"Simplified Stop Times: {len(simplified_stop_times)} rows remaining.")
    
    return stop_times_copy

In [97]:
def create_graph_with_schedule(stop_times, stops, trips):
    

    
    graph = defaultdict(list)
    
    stop_id_to_name = stops.set_index("stop_id")["stop_name"].to_dict()
    trip_id_to_route = trips.set_index("trip_id")["route_id"].to_dict()

    # Sortiere stop_times nach Trip und Stop-Sequence
    stop_times = stop_times.sort_values(by=["trip_id", "stop_sequence"])
    grouped = stop_times.groupby("trip_id")

    for trip_id, group in grouped:

        stops_in_trip = group["stop_id"].tolist()
        arrival_times = group["arrival_time"].tolist()
        departure_times = group["departure_time"].tolist()

        # Füge Verbindungen zwischen aufeinanderfolgenden Haltestellen hinzu
        for i in range(len(stops_in_trip) - 1):
            start_stop_id = stops_in_trip[i]
            end_stop_id = stops_in_trip[i + 1]

            start_departure = time_to_minutes(departure_times[i])
            end_arrival = time_to_minutes(arrival_times[i + 1])

            travel_time = end_arrival - start_departure  # Dauer in Minuten

            if travel_time > 0:  # Vermeide ungültige Zeiten
                start_stop_name = stop_id_to_name[start_stop_id]
                end_stop_name = stop_id_to_name[end_stop_id]
                route_id = trip_id_to_route[trip_id]  # Hole die Route/Linie

                graph[start_stop_name].append((end_stop_name, start_departure, end_arrival, route_id))

    return graph

In [98]:
# Define input parameters
start_stop_name = "Schattendorf Kirchengasse"
end_stop_name = "Flughafen Wien Bahnhof"
start_datetime = "2024-10-16 14:30:00"
time_budget_minutes = 120  # 2 hours

# Run the `prepare_network` function
simplified_stop_times = prepare_network(
    start_stop_name,
    end_stop_name,
    start_datetime,
    time_budget_minutes,
    stop_times,
    trips,
    calendar,
    calendar_dates,
    stops
)

# Output the simplified stop_times DataFrame
print(simplified_stop_times)

Rows before time window filter: 315698
Rows after time window filter: 32584
Rows before availability filter: 32584
Rows after availability filter: 19897
Simplified Stop Times: 19897 rows remaining.
                         trip_id arrival_time departure_time          stop_id  \
203        1.TA.1-S8-T-j24-1.1.R     14:33:00       14:33:00   at:47:1169:0:3   
204        1.TA.1-S8-T-j24-1.1.R     14:38:00       14:38:00   at:47:1170:0:8   
340       10.TA.1-S3-K-j24-1.5.R     16:00:00       16:00:00   at:42:3844:0:1   
341       10.TA.1-S3-K-j24-1.5.R     16:05:00       16:05:00   at:42:3843:0:2   
342       10.TA.1-S3-K-j24-1.5.R     16:08:00       16:08:00   at:42:3847:0:1   
...                          ...          ...            ...              ...   
315604  39.TA.32-592-0-j24-1.3.H     16:15:00       16:15:00   at:46:3040:0:6   
315605   4.TA.32-592-0-j24-1.1.R     15:45:00       15:45:00   at:46:3040:0:6   
315650  53.TA.32-592-0-j24-1.4.H     16:15:00       16:15:00  at:42:3642:

In [99]:
# # Fix für Dijkstra-Algorithmus mit Umstiegsoptimierung
# def dijkstra_with_reliability_fixed(graph, start_name, end_name, start_time_minutes):
#     pq = [(start_time_minutes, start_name, [], 1.0, None)]
#     visited = set()

#     while pq:
#         current_time, current_stop, path, reliability, last_route = heapq.heappop(pq)

#         if (current_stop, current_time) in visited:
#             continue
#         visited.add((current_stop, current_time))

#         path = path + [(current_stop, current_time)]
#         for neighbor, departure_time, arrival_time, route_id in graph[current_stop]:
#             if departure_time >= current_time:
#                 transfer_reliability = 1.0 if last_route == route_id else calculate_transfer_probability(arrival_time, departure_time)

#                 new_current_time = arrival_time
#                 new_reliability = reliability * transfer_reliability
#                 heapq.heappush(pq, (
#                     new_current_time, neighbor, path + [(route_id, departure_time, arrival_time)], new_reliability, route_id))

#         if current_stop == end_name:
#             return current_time, path, reliability

#     return float("inf"), [], 0.0

In [100]:
def dijkstra_with_reliability_fixed(graph, start_name, end_name, start_time_minutes, time_budget_minutes,
                                    exclude_routes=set()):
    pq = [(start_time_minutes, start_name, [], 1.0,
           None)]  # (aktuelle Zeit, aktuelle Haltestelle, Pfad, Zuverlässigkeit, letzte Linie)
    visited = set()
    MIN_TRANSFER_TIME = 5  # Mindestumstiegszeit in Minuten

    while pq:
        current_time, current_stop, path, reliability, last_route = heapq.heappop(pq)

        if (current_stop, current_time) in visited:
            continue
        visited.add((current_stop, current_time))

        path = path + [(current_stop, current_time)]

        if current_time - start_time_minutes > time_budget_minutes:
            continue  # Pruning: Abbruch, wenn Zeitbudget überschritten

        for neighbor, departure_time, arrival_time, route_id in graph[current_stop]:
            if departure_time >= current_time and route_id not in exclude_routes:
                # Prüfe, ob es ein Umstieg ist (Linienwechsel)
                is_transfer = last_route is not None and last_route != route_id
                if is_transfer:
                    # Mindestumstiegszeit von 5 Minuten nur bei Umstiegen
                    transfer_time = departure_time - current_time
                    if transfer_time < MIN_TRANSFER_TIME:
                        continue  # Pruning: Zu wenig Zeit für Umstieg

                # Berechne neue Zuverlässigkeit mit Transferwahrscheinlichkeit
                if not is_transfer:  # Keine Zuverlässigkeitsänderung bei gleicher Linie
                    transfer_reliability = 1.0
                else:
                    transfer_reliability = calculate_transfer_probability(transfer_time)

                new_current_time = arrival_time
                new_reliability = reliability * transfer_reliability

                heapq.heappush(pq, (
                    new_current_time, neighbor, path + [(route_id, departure_time, arrival_time)], new_reliability,
                    route_id))

        if current_stop == end_name:
            return current_time, path, reliability

    return float("inf"), [], 0.0  # Keine Route gefunden

The transfer probability is calculated using cdf, but we say there are not depature delays. If there are, reliability only goes higher, for all itineraries, we want the most extreme case however. Furthermore, it is simply easier to calculate. 

In [101]:
#BACK UPS NOT IM

# Hauptprogramm
if __name__ == "__main__":
    start_stop_name = "Schattendorf Kirchengasse"
    end_stop_name = "Flughafen Wien Bahnhof"
    start_datetime = "2024-10-16 14:30:00"
    time_budget_minutes = 120

    #agency, stops, routes, trips, stop_times, calendar, calendar_dates = import_data()

    start_time_obj = datetime.strptime(start_datetime, "%Y-%m-%d %H:%M:%S")
    date = start_time_obj.date()
    start_time_minutes = start_time_obj.hour * 60 + start_time_obj.minute

    simplified_stop_times = prepare_network(
        start_stop_name, 
        end_stop_name, 
        start_datetime, 
        time_budget_minutes, 
        stop_times, 
        trips, 
        calendar, 
        calendar_dates, 
        stops
    )

    graph = create_graph_with_schedule(simplified_stop_times, stops, trips)
    print(f"Start Stop: {start_stop_name}, End Stop: {end_stop_name}")
    print(f"Graph Nodes: {list(graph.keys())}")

    if start_stop_name not in graph or end_stop_name not in graph:
        print("🚨 Ungültige Start- oder Zielhaltestelle!")
        sys.exit()

    # Finde optimierte Route
    arrival_time_minutes_fixed, path_fixed, reliability_fixed = dijkstra_with_reliability_fixed(
        graph, start_stop_name, end_stop_name, start_time_minutes, time_budget_minutes
    )

    # Ergebnis anzeigen
    if arrival_time_minutes_fixed < float("inf"):
        arrival_time_fixed = minutes_to_time(arrival_time_minutes_fixed)
        print(f"\n📍 Optimierte zuverlässigste Route von {start_stop_name} nach {end_stop_name}:")

        last_route = None
        grouped_routes = []

        for i in range(1, len(path_fixed) - 1, 2):
            current_stop, current_time = path_fixed[i-1]
            route_id, departure_time, arrival_time = path_fixed[i]
            next_stop, _ = path_fixed[i + 1]

            if route_id == last_route:
                grouped_routes[-1]["stops"].append((next_stop, arrival_time))
            else:
                grouped_routes.append({
                    "route_id": route_id,
                    "start_stop": current_stop,
                    "departure_time": departure_time,
                    "stops": [(next_stop, arrival_time)]
                })
            last_route = route_id

        for segment in grouped_routes:
            start = segment["start_stop"]
            dep_time = minutes_to_time(segment["departure_time"])
            route = segment["route_id"]
            stops = " → ".join([f"{stop} (Ankunft: {minutes_to_time(arr)})" for stop, arr in segment["stops"]])
            print(f"  🚆 {start} (Abfahrt: {dep_time}) → {stops} mit Linie {route}")

        print(f"\n🎯 Endstation: {end_stop_name} (Ankunft: {arrival_time_fixed})")
        print(f"🔹 Gesamt-Zuverlässigkeit der Route: {reliability_fixed:.2f}\n")
    else:
        print(f"\n⚠️ Keine zuverlässige Route von {start_stop_name} nach {end_stop_name} gefunden.\n")

Rows before time window filter: 315698
Rows after time window filter: 32584
Rows before availability filter: 32584
Rows after availability filter: 19897
Simplified Stop Times: 19897 rows remaining.
Start Stop: Schattendorf Kirchengasse, End Stop: Flughafen Wien Bahnhof
Graph Nodes: ['Wörgl Süd-Bruckhäusl Bahnhof', 'Landeck-Zams Bahnhof', 'St. Anton a. A. Bahnhof', 'Bludenz Bahnhof', 'Villach Hauptbahnhof', 'Flughafen Wien Bahnhof', 'Lienz Bahnhof', 'Sillian Bahnhof', 'Krems Bahnhof', 'Gedersdorf Bahnhof', 'Nickelsdorf Bahnhof', 'Zurndorf Bahnhof', 'Parndorf Bahnhof', 'Parndorf Ort Bahnhof', 'Bruck/Leitha Bahnhof', 'Götzendorf Bahnhof', 'Wien Grillgasse', 'Wöllersdorf Bahnhof', 'Feuerwerksanstalt Bahnhof', 'Bad Fischau Bahnhof', 'Bad Fischau-Brunn Bahnhof', 'Wien Heiligenstadt', 'Wien Spittelau', 'Kleinreifling Bahnhof', 'Kastenreith Bahnhst', 'Küpfern Bahnhst', 'Großraming Bahnhof', 'Reichraming Bahnhof', 'Losenstein', 'Innsbruck Hauptbahnhof', 'Innsbruck Messe Bahnhof', 'Rum Bahnhof',